In [2]:
!pip install numpy pandas matplotlib yfinance sklearn tensorflow prophet streamlit

ERROR: Could not find a version that satisfies the requirement sklearn (from versions: none)
ERROR: No matching distribution found for sklearn


In [3]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install prophet


Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import date
from prophet import Prophet
import streamlit as st

In [7]:
# Constants
START_DATE = "2015-01-01"
TODAY_DATE = date.today().strftime("%Y-%m-%d")


In [8]:
# Sidebar for user input
st.sidebar.header("Input Parameters")
selected_stock = st.sidebar.text_input("Enter Stock Ticker (e.g., AAPL)", "AAPL")
prediction_days = st.sidebar.slider("Prediction Days (for LSTM)", min_value=30, max_value=365, value=60, step=10)
forecast_days = st.sidebar.slider("Forecast Period (for FBProphet)", min_value=30, max_value=365, value=90, step=10)


2024-12-12 23:52:50.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:52:50.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [10]:
# Fetch stock data
def load_data(ticker):
    data = yf.download(ticker, START_DATE, TODAY_DATE)
    data.reset_index(inplace=True)
    return data

st.write(f"Fetching data for {selected_stock}...")
try:
    data = load_data(selected_stock)
    st.write("Data Loaded Successfully")

    # Display raw data
    st.subheader("Raw Data")
    st.write(data.tail())

    # Prepare data for LSTM
    data_lstm = data[['Date', 'Close']]
    data_lstm['Date'] = pd.to_datetime(data_lstm['Date'])
    data_lstm.set_index('Date', inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_lstm)

    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    def create_dataset(data, time_step=1):
        X, y = [], []
        for i in range(len(data) - time_step - 1):
            X.append(data[i:(i + time_step), 0])
            y.append(data[i + time_step, 0])
        return np.array(X), np.array(y)

    time_step = 60
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Build LSTM Model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1)

    # Predict
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    # Plot LSTM Predictions
    test_dates = data_lstm.index[train_size + time_step + 1:]
    st.subheader("LSTM Prediction Plot")
    fig, ax = plt.subplots()
    ax.plot(test_dates, data_lstm['Close'][train_size + time_step + 1:], label='Actual Price')
    ax.plot(test_dates, predictions, label='Predicted Price')
    ax.legend()
    st.pyplot(fig)

    # Prepare data for FBProphet
    df_fb = data[['Date', 'Close']]
    df_fb.rename(columns={"Date": "ds", "Close": "y"}, inplace=True)

    # Forecasting with FBProphet
    m = Prophet()
    m.fit(df_fb)

    future = m.make_future_dataframe(periods=forecast_days)
    forecast = m.predict(future)

    # Plot FBProphet Forecast
    st.subheader("FBProphet Forecast Plot")
    fig1 = m.plot(forecast)
    st.write(fig1)

    st.subheader("Forecast Components")
    fig2 = m.plot_components(forecast)
    st.write(fig2)

except Exception as e:
    st.error(f"An error occurred: {e}")

2024-12-12 23:58:17.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
[*********************100%***********************]  1 of 1 completed
2024-12-12 23:58:17.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:17.615 Thread 'MainThread': miss

Epoch 1/10
31/31 [==============================] - 11s 97ms/step - loss: 0.0107
Epoch 2/10
31/31 [==============================] - 2s 70ms/step - loss: 4.6260e-04
Epoch 3/10
31/31 [==============================] - 2s 69ms/step - loss: 3.0598e-04
Epoch 4/10
31/31 [==============================] - 2s 68ms/step - loss: 2.9574e-04
Epoch 5/10
31/31 [==============================] - 2s 66ms/step - loss: 2.8821e-04
Epoch 6/10
31/31 [==============================] - 2s 73ms/step - loss: 2.9019e-04
Epoch 7/10
31/31 [==============================] - 2s 64ms/step - loss: 2.7943e-04
Epoch 8/10
31/31 [==============================] - 2s 69ms/step - loss: 2.6908e-04
Epoch 9/10
31/31 [==============================] - 2s 73ms/step - loss: 2.7626e-04
Epoch 10/10
14/14 [==============================] - 1s 17ms/step


2024-12-12 23:58:54.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:54.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:54.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:55.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 23:58:55.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\user\AppData\Local\Temp\ipykernel_18324\519651843.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-12-12 23:58:55.834 Thread 'MainThread': missing ScriptRunContext! 